In [1]:
import sys

sys.path.append("../")

import torch
import torch.utils as utils

import pytorch_lightning as pl

from torchemlp.groups import SO, O, S, Z
from torchemlp.nn.equivariant import EMLP
from torchemlp.nn.utils import RegressionLightning
from torchemlp.datasets import Inertia

In [2]:
TRAINING_SET_SIZE = 1000
# TRAINING_SET_SIZE = 10_000
BATCH_SIZE = 512

# N_EPOCHS = int(900000 / TRAINING_SET_SIZE)
N_EPOCHS = 10

DL_WORKERS = 0
# DL_WORKERS = 8

N_CHANNELS = 384
N_LAYERS = 3

In [3]:
dataset = Inertia(TRAINING_SET_SIZE, device="cuda")
# dataset = Inertia(TRAINING_SET_SIZE, device="cpu")

G = SO(3)

f"Input type: {dataset.repin(G)}, output type: {dataset.repout(G)}"

'Input type: 5V⁰+5V, output type: V²'

In [4]:
val_size = 250
test_size = 250
train_size = len(dataset) - val_size - test_size

split_data = utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = utils.data.DataLoader(
    split_data[0], batch_size=BATCH_SIZE, num_workers=DL_WORKERS, shuffle=True
)
val_loader = utils.data.DataLoader(
    split_data[1], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)
test_loader = utils.data.DataLoader(
    split_data[2], batch_size=BATCH_SIZE, num_workers=DL_WORKERS
)

In [5]:
model = EMLP(dataset.repin, dataset.repout, G, N_CHANNELS, N_LAYERS).cuda()
# model = EMLP(dataset.repin, dataset.repout, G, N_CHANNELS, N_LAYERS)
plmodel = RegressionLightning(model)

In [6]:
model.repin

5V⁰+5V

In [7]:
trainer = pl.Trainer(
    limit_train_batches=BATCH_SIZE,
    max_epochs=N_EPOCHS,
    accelerator="gpu"
    # limit_train_batches=BATCH_SIZE, max_epochs=N_EPOCHS, accelerator="cpu"
)
trainer.fit(plmodel, train_loader, val_loader)
trainer.test(plmodel, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | EMLP | 583 K 
-------------------------------
583 K     Trainable params
0         Non-trainable params
583 K     Total params
2.335     Total estimated 

/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (1) is smaller t

Epoch 0:  50%|██████████████████████████████████████████████████████████████████████████                                                                          | 1/2 [00:00<00:00,  1.73it/s, loss=31.5, v_num=11]
Validation: 0it [00:00, ?it/s]
Epoch 1:  50%|██████████████████████████████████████████████████████████████████████████                                                                          | 1/2 [00:00<00:00,  1.69it/s, loss=31.5, v_num=11]
Validation: 0it [00:00, ?it/s]
Epoch 2:  50%|██████████████████████████████████████████████████████████████████████████                                                                          | 1/2 [00:00<00:00,  1.79it/s, loss=31.4, v_num=11]
Validation: 0it [00:00, ?it/s]
Epoch 3:  50%|██████████████████████████████████████████████████████████████████████████                                                                          | 1/2 [00:00<00:00,  1.63it/s, loss=31.3, v_num=11]
Validation: 0it [00:00, ?it/s]
Epoch 4:  50%|██████

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.04it/s, loss=27.2, v_num=11]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/rytse/mambaforge/envs/l2e10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.38it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    12.942802429199219     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 12.942802429199219}]